<a href="https://colab.research.google.com/github/palVikram/Advance_tensorflow/blob/main/NLP_tensorflow/IMDB_review_word_embedding" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

imdb, info=tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteELNGXA/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteELNGXA/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
import numpy as np

train_data, test_data= imdb['train'], imdb['test']

In [6]:
training_sentence=[]
training_labels=[]

testing_sentence=[]
testing_labels=[]

for s,l in train_data:
  training_sentence.append(str(s.numpy()))
  training_labels.append(l.numpy())

for s,l in test_data:
  testing_sentence.append(str(s.numpy()))
  testing_labels.append(l.numpy()) 

In [11]:
#training_labels

In [8]:
training_labels_final=np.array(training_labels)
testing_labels_final=np.array(testing_labels)

In [9]:
### config
vocab_size=10000
embedding_dims=16
max_length=120
truc_type="post"
oov_tok='<OOV>'

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [23]:
tokenizer=Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentence)

word_index=tokenizer.word_index

sequences=tokenizer.texts_to_sequences(training_sentence)
padded=pad_sequences(sequences, maxlen=max_length, truncating=truc_type)

testing_sequences=tokenizer.texts_to_sequences(testing_sentence)
testing_padded=pad_sequences(testing_sequences, maxlen=max_length, truncating=truc_type)

In [24]:
model=tf.keras.Sequential([
          tf.keras.layers.Embedding(vocab_size, embedding_dims, input_length=max_length),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(6, activation='relu'),
          tf.keras.layers.Dense(1, activation='sigmoid')                       
])

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 120, 16)           160000    
                                                                 
 flatten_2 (Flatten)         (None, 1920)              0         
                                                                 
 dense_2 (Dense)             (None, 6)                 11526     
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded, training_labels_final, epochs=10, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2401 - accuracy: 0.9063 - val_loss: 0.4143 - val_accuracy: 0.8180
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1033 - accuracy: 0.9714 - val_loss: 0.5013 - val_accuracy: 0.8070
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0269 - accuracy: 0.9964 - val_loss: 0.6049 - val_accuracy: 0.8052
Epoch 4/10
782/782 [==============================] - 4s 6ms/step - loss: 0.0066 - accuracy: 0.9995 - val_loss: 0.6915 - val_accuracy: 0.8024
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.7467 - val_accuracy: 0.8066
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 8.6108e-04 - accuracy: 1.0000 - val_loss: 0.8000 - val_accuracy: 0.8066
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 4.6437e-04 - accuracy: 1.0000 - val_loss: 0.8465 - val_accuracy: 0.807

In [28]:
e=model.layers[0]
weights=e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [30]:
import io
reversed_word_index=tokenizer.index_word
out_v=io.open('vecs.tsv', 'w', encoding='utf-8')
out_m=io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
  word=reversed_word_index[word_num]
  embddings=weights[word_num]
  out_m.write(word+'\n')
  out_v.write('\t'.join([str(x) for x in embddings])+'\n')
out_v.close()
out_m.close()

In [31]:
try:
  from google.colab import files
except ImportError:
  pass

else:
  files.download("vecs.tsv")
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
### https://projector.tensorflow.org/
### Visulaizing word embedding using the above link 
### upload the downloaded TSV files on embedding projector to visulaize